In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys


sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

from llm_lexical_simplifier import LLMLexicalSimplifier
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm import tqdm

Mounted at /content/drive


# LLM based

Install this and the restart the session. This is a workaround for a known bug in flash-attn. Do not re-run this cell after restarting the session.

In [2]:
#!pip install accelerate flash-attn

In [3]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    # Flash attention is only supported starting Ampere architecture
    # So we cant use it on free-tier GPUs
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

# Define PAD Token = EOS Token
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [13]:
de_messages = [
    {"role": "user", "content": "Der Maschinenbau hat in Europa durch die Bildung der EU eine starke Erleichterung erhalten. Die vereinfachte Version des vorigen Satzes ist: Der Maschinenbau hat in Europa durch die Bildung der EU eine starke [MASK] erhalten. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Vereinfachung", "Entspannung", "Begünstigung", "Unterstützung", "Rückenwind,  "Förderung", "Rückhalt", "Aufwind", "Antrieb", "Erleichterung"]'},
    {"role": "user", "content": "Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen Sanktus signalisiert. Die vereinfachte Version des vorigen Satzes ist: Die EU-Renaturierungsverordnung galt eigentlich bereits als durchgebracht; das EU-Parlament hatte zugestimmt und die Staaten ihren grundsätzlichen [MASK] signalisiert. Ten results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["Zustimmung", "Einverständnis", "Okay", "Segen", "Billigung, Bestätigung, Bejahung, Zuspruch, Akzeptanz, Befürwortung"]'},
]
en_messages = [
    {"role": "user", "content": "The president’s relatives were also said to be critical of the way his closest advisers had prepared him for the debate. The simplified version of the previous sentence is: The president’s relatives were also said to be critical of the way his closest [MASK] had prepared him for the debate. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["staff", "team", "aides", "consultants", "assistants", "strategists", "supporters", "handlers", "counselors", "staffers"]'},
    {"role": "user", "content": "The intermediate sprint will come 94km or so into the stage. The simplified version of the previous sentence is: The intermediate sprint will come 94km or so into the [MASK]. Results as ordered, AST parsable Python list:"},
    {"role": "assistant", "content": '["segment", "part", "section", "phase", "leg", "interval", "course", "route", "portion", "strech"]'},
]

llm_ls = LLMLexicalSimplifier(model=model, tokenizer=tokenizer,
                              pattern='{original_sentence}. Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}. Ten results as ordered, AST parsable Python list:',
                              exemplars=en_messages, mask_token='[MASK]', generation_args=generation_args)

Using cuda for model inference.
Using mask token: "[MASK]".


In [14]:
llm_output = llm_ls.generate_substitutions_for('heikel', 'Die politische Lage in Syrien ist heikel.')
llm_output

['heikel',
 'kompliziert',
 'schwierig',
 'risikabel',
 'unstabil',
 'bedrohlich',
 'gefährlich',
 'kränklich',
 'problematisch',
 'schwierig']

In [15]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(llm_ls, {
        Language.EN: {'pattern': '{original_sentence}. The simplified version of the previous sentence is: '
                                 '{sentence_with_complex_word_masked}. Ten results as ordered, AST parsable '
                                 'Python list:',
                      'exemplars': en_messages}})
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Benchmarking:   0%|          | 0/929 [00:00<?, ?it/s]

found is set this way when the FOR loop exits ; inside the execution of the loop , found is not modified by the FOR statement , although it may be changed by the execution of other statements within the loop body . execution


Benchmarking:   0%|          | 1/929 [00:05<1:26:04,  5.56s/it]

Failed to parse the output from the LLM: The provided string is not a valid list representation:  ["execution", "iteration", "cycle", "iteration", "iteration", "iteration", "iteration", "iteration", "iteration", "iteration"]

"although it may be changed by the [MASK] of other statements within the loop body"

Results as ordered, AST parsable Python list:

["execution", "execution", "execution", "execution", "execution", "execution", "execution", "execution",Returning empty list.
Alexander Stepanovich Popov was a Russian physicist who first demonstrated the practical application of electromagnetic waves , although he did not apply for a patent for his invention . demonstrated


...

in Wales , four Church of England dioceses were disestablished in 1920 , becoming separated from the Church of England in the process and subsequently becoming the Church in Wales . separated


Benchmarking:   0%|          | 4/929 [00:15<56:53,  3.69s/it]

Ge_lson de Carvalho Soares or simply Ge_lson , is a striker . striker


Benchmarking:   0%|          | 4/929 [00:17<1:09:03,  4.48s/it]


KeyboardInterrupt: 

#BERT based

In [ ]:
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
bert_ls = SimpleBertLexicalSimplifier(model, tokenizer, bert_pattern, None)

Using cuda for model inference.
No exemplars provided, using zero-shot mode.


In [ ]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(bert_ls, {Language.EN: '{original_sentence} The simpler version of the previous sentence is: {sentence_with_complex_word_masked}',
                                 Language.DE: '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'})
suite.run()

KeyboardInterrupt: 